**Introduction**

Consider the non-dimensionalized time-dependent Schrodinger equation (TDSE):

$\displaystyle i \partial_t \psi = \mathcal{H} \psi$.

In this notebook, we work in one spatial dimension, so the kinetic operator is

$\displaystyle \mathcal{K} = -\frac{1}{2} \partial_{xx}$.

In real space, the potential operator $\mathcal{V}$ works like this:

$\displaystyle (\mathcal{V} \psi)(x,t) = v(x) \psi(x,t)$,

where $v(x)$ is a real potential energy function.

The Hamiltonian is $\mathcal{H} = \mathcal{K} + \mathcal{V}$.  Clearly $\mathcal{H}$ is self-adjoint or Hermitian.

**Forward problem**

In the forward problem, $V$ and $\psi(x,0)$ are specified and the task is to solve for $\psi(x,t)$ for $t > 0$.

We will do this numerically through a basis function method.  Our spatial domain is the closed interval $[-a,a]$.

Let $\{ \phi_n(x) \}_{n=-\infty}^\infty$ be a collection of basis functions.  We take these functions to be elements of the Hilbert space $L^2([-a,a]; \mathbb{C})$.  This Hilbert space consists of complex-valued, square-integrable functions with domain $[-a,a]$.  The inner product in this space is

$\displaystyle (f, g) = \int_{x=-a}^a \overline{f(x)} g(x) \, dx$,

and therefore the norm is

$\displaystyle \| f \| = (f, f)^{1/2} = \left( \int_{x=-a}^a |f(x)|^2 \, dx \right)^{1/2}$.

We assume the basis is $orthonormal$ and $complete$ in $L^2([-a,a]; \mathbb{C})$.  Orthonormality means that

$\displaystyle (\phi_m, \phi_n) = \delta_{mn} = \begin{cases} 1 & m = n \\ 0 & m \neq n. \end{cases}$

Completeness means that for any function $f \in L^2([-a,a]; \mathbb{C})$, there exists an infinite sequence of coefficients $\{ \alpha_n \}_{n=-\infty}^\infty$ such that when we form the expansion

$\displaystyle g_N(x) = \sum_{n=-N}^N \alpha_n \phi_n(x)$,

the expansion $converges$ to $f$ as $N \to \infty$, i.e., $\lim_{N \to \infty} \| f - g_N \| = 0$.  Or, to put it another way, we can represent any $f$ (in our Hilbert space) arbitrarily well by expanding $f$ as a linear combination of basis functions.

Using this complete, orthonormal basis, we can represent the kinetic and potential operators as matrices:

$\displaystyle K_{mn} = (\phi_m, \mathcal{K} \phi_n) = \int_{x=-a}^a \overline{\phi_m(x)} (\mathcal{K} \phi_n)(x) \, dx = -\frac{1}{2} \int_{x=-a}^a \overline{\phi_m(x)} \partial_{xx} \phi_n(x) \, dx$

Note that if we wanted to we could write

$\displaystyle K_{mn} = -\frac{1}{2} \overline{\phi_m(x)} \partial_x \phi_n(x) \, dx \biggr|_{x=-a}^{a} + \frac{1}{2} \int_{x=-a}^a \partial_{x} \overline{\phi_m(x)} \partial_{x} \phi_n(x) \, dx$,

which shows that if the boundary term vanishes, $K$ must be a Hermitian matrix.

$\displaystyle V_{mn} = (\phi_m, \mathcal{V} \phi_n) = \int_{x=-a}^a \overline{\phi_m(x)} (\mathcal{V} \phi_n)(x) \, dx = \int_{x=-a}^a \overline{\phi_m(x)} v(x) \phi_n(x) \, dx$

In this notebook, we will consider an orthonormal basis that *diagonalizes* the kinetic operator.  This basis is the Fourier basis in complex exponential form:

$\phi_n(x) = (2 a)^{-1/2} e^{i \pi n x/a}$.

We can check that, for integers $n$ and $m$ such that $n \neq m$,

$\displaystyle (\phi_m, \phi_n) = \frac{1}{2a} \int_{x=-a}^a e^{i \pi (n-m) x/a} \, dx = \frac{1}{2a} \frac{a}{i \pi (n-m)} \left( e^{i \pi (n-m)} - e^{-i \pi (n-m)} \right) = 0$.

For integer $n$, it is clear that $(\phi_n, \phi_n) = 1$, completing our check of orthonormality.

Completeness follows from the Riesz-Fischer theorem.

Now we see that for the kinetic operator,

$\displaystyle (\mathcal{K} \phi_n)(x) = \frac{ \pi^2 n^2 }{ 2 a^2 } \phi_n(x) $.

Hence by orthonormality,

$\displaystyle K_{mn} = (\phi_m, \mathcal{K} \phi_m) = \frac{ \pi^2 n^2 }{ 2 a^2 } \delta_{mn} $,

implying that $K$ is a diagonal matrix.  As for the potential operator $\mathcal{V}$, we see that

$\displaystyle V_{mn} = \frac{1}{2a} \int_{x=-a}^a e^{i \pi (n-m) x/a} v(x) \, dx$.

Clearly $V_{mn}$ is constant for all $(m,n)$ such that $n-m$ is constant.  This means that the matrix $V$ is Toeplitz, i.e., constant along diagonals.  Additionally, we see that $V_{mn}$ is equivalent to a *Fourier series coefficient* of the function $v(x)$.  We can compute this coefficient using the FFT.

**Quick points on potentials, symmetry, and periodicity**

What happens for an *even* or *symmetric* potential $v(x)$?  We start with

$\displaystyle \theta_{-j} = (2a)^{-1/2} \int_{x=-a}^a e^{i \pi j x/a} v(x) \, dx$,

and make the substitution $x = -z$ in this integral.  Chasing down all the minus signs, we obtain

$\displaystyle \theta_{-j} = (2a)^{-1/2} \int_{z=a}^{-a} e^{-i \pi j z/a} v(-z) \, (-dz)$.

We use the minus sign from $-dz$ to flip the limits of integration, and we use $v(-z) = v(z)$.  Then we obtain

$\displaystyle \theta_{-j} = (2a)^{-1/2} \int_{z=-a}^{a} e^{-i \pi j z/a} v(z) \, dz$ = \theta_j$.

Hence $\theta_j = \overline{\theta_j}$.  We have shown that for an *even* potential $v(x)$, the Fourier coefficients will be real.

Something we have not yet discussed is periodicity.  We write our potential function as

$\displaystyle v(x) = \sum_{j=-2N}^{j=2N} \theta_j \phi_j(x)$.

By virtue of the Fourier basis we have chosen, this potential function will be periodic with period $2a$ on the interval $[-a,a]$.  This is because, for all $j$, we have

$\displaystyle \phi_n(x + 2a) = (2 a)^{-1/2} e^{i \pi n (x+2a)/a} =  (2 a)^{-1/2} e^{i \pi n x/a} e^{2 \pi i n} = \phi_n(x)$.

The same is true for $\psi(x,t)$; it will also be $2a$-periodic on $[-a,a]$.

We justify this based on the principle that if we choose $a$ sufficiently large, and if our initial wave function is localized in space near $x=0$, then the wave function's magnitude will decay (as a function of spatial coordinate x) to be close to zero near the boundaries $x= \pm a$.  Hence the behavior of the potential beyond the boundaries $x = \pm a$ is of no practical concern to us.

When we generate data using our forward method, we must monitor our wave function and check that during the time interval of interest, its magnitude stays close to zero near the boundaries.  If this isn't true, we need to rerun the simulation with larger $a$.
